# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** There are two types of supervised learning described in this unit. The first are regression problems which have continuous or near continous response variables. The second are classification problems which require identification of which set of categories a new observation belongs to. 

I believe the project for student intervention is a classification supervised learning problem as the response variable is categorical. The set of students can be labeled "passed" with either a value of "yes" or "no". 

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [9]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [13]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]-1

# TODO: Calculate passing students
n_passed = student_data[student_data['passed'] == "yes"].shape[0]

# TODO: Calculate failing students
n_failed = student_data[student_data['passed'] == "no"].shape[0]

# TODO: Calculate graduation rate
grad_rate = float(n_passed) / (n_students) *100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [14]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [15]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [16]:
from sklearn.cross_validation import train_test_split

# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train


# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, 
                                                                             random_state=5)
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: ** I have decided to go with **SVM, Naive Bayes, and Gradient Tree Boosting** as my three supervised learning models.

**1) SVM(Support Vector Machines):** The general applications of SVM include regression, classification, and outlier detection. This classifier works by taking labeled training data and tries to categorize it by using an optimal hyperplane. 

Strengths: - Effective in high dimensional spaces and when # dimensions is greater than # of samples. They work well with both dense and sparse data.

- They use support vectors as a subset of training points in the decision function and thus are memory efficient.

- They are versatile as different kerneling functions can be applied to the decision function (Even possible to make custom kernels).

Weaknesses: 

- If the number of features is greater than the number of samples then the classifier has poorer performance.

- Prone to overfit the features of the data.

I decided on SVM because there is a high number of features (less than samples) in the data set that we are working with and for all the advantageous reasons stated above - versatile, effectivity in high dimensional space, and memory efficiency. 

**2) Naive Bayes:** As mentioned during the unit, Naive Bayes can be applied to document classification and spam filtering. This classifier works by using Baye's Theorem (an equation that calculates likelihood of hypothesis based on evidence/prior probability) to try and classify data.

Strengths: 

- Requires only a small training set to estimate necessary parameters.

- Faster to train and fast to classify.

- Not sensitive to irrelevant features.

Weaknesses: 

- Makes an assumption that all features are independant which is not necessarily true.

I chose Naive Bayes because of its speed.

**3) Gradient Tree Boosting:** GBT is a classifier that works by using decision trees of fixed size and assigning scores to the leaf nodes which affect their contribution to the output - these scores are found using the gradient descent algorithm. The general applications for GBT are various, but one prominent one is Web Search Ranking.

Strengths: 

- Natural handling of heterogenous (mixed type) features
- High accuracy of predictions
- Robust from the impact of outliers

Weaknesses: 

- Lack of scalability (sequential boosting cannot be parallelized) 

- Slow execution time

I chose GBT because of its high predicativity, and strength in handling heterogenous features. 



### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [18]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [24]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from sklearn.metrics import f1_score

# from sklearn import model_B
# from skearln import model_C

# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = GradientBoostingClassifier()
clf_C = svm.SVC()

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
print "Naive Bayes:"
train_predict(clf_A,X_train_100,y_train_100,X_test,y_test)
train_predict(clf_A,X_train_200,y_train_200,X_test,y_test)
train_predict(clf_A,X_train_300,y_train_300,X_test,y_test)

print "Gradient Boosting Classifier:"
train_predict(clf_B,X_train_100,y_train_100,X_test,y_test)
train_predict(clf_B,X_train_200,y_train_200,X_test,y_test)
train_predict(clf_B,X_train_300,y_train_300,X_test,y_test)

print "Support Vector Machines:"
train_predict(clf_C,X_train_100,y_train_100,X_test,y_test)
train_predict(clf_C,X_train_200,y_train_200,X_test,y_test)
train_predict(clf_C,X_train_300,y_train_300,X_test,y_test)

Naive Bayes:
Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0017 seconds
Made predictions in 0.0006 seconds.
F1 score for training set: 0.8447.
Made predictions in 0.0005 seconds.
F1 score for test set: 0.7639.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0007 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8310.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7132.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0008 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 0.8274.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7669.
Gradient Boosting Classifier:
Training a GradientBoostingClassifier using a training set size of 100. . .
Trained model in 0.0823 seconds
Made predictions in 0.0007 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7647.
Training a GradientBo

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Naive Bayes**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |  0.0006                 |        0.0005          |       0.8447     |        0.7639   |
| 200               |        0.0003           |        0.0002          |       0.8310     |        0.7132   |
| 300               |        0.0004           |        0.0003          |       0.8274     |    0.7669       |

** Classifer 2 - Gradient Boosting Classifier**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0007              |        0.0010          |       1.0000     |        0.7647   |
| 200               |     0.0007              |        0.0004          |       0.9892     |        0.7737   |
| 300               |     0.0011              |        0.0007          |       0.9667     |     0.7634      |

** Classifer 3 - Support Vector Machines**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0007            |        0.0007          |       0.8519     |      0.7898     |
| 200               |       0.0023            |        0.0012          |       0.8616     |      0.7867     |
| 300               |       0.0064            |        0.0028          |       0.8764     |      0.7660     |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** Analyzing the tabular results of the three different classifiers, I conclude that the best model for the student intervention system would be the SVM. We are looking for accuracy in our classification on whether or not a student requires further assistance to ensure we can help the majority of them before it is too late. The SVM classifier boasted the highest F1 score out of the three that I chose and so it had the highest testing accuracy. This will allow us to help the most amount of students which do need help. Although the prediction time for the training and test set for SVN was the longest, it was not far off from the time taken from the other two classifiers. I believe it is better to have higher testing accuracy than computation time and so SVN is my choice.



### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: ** A Support Vector Machine seperates the data with a line (or hyperplane) into a group of successful and unsuccessful students. The classifier does this in a way that maximizes the distance from the line to the nearest points of each group as seen in the picture below. The support vectors themselves are the data points that lie closest to the decision surface. They have a direct bearing on the optimum location of the decision surface.


<img src="SVM.png">


When there are no clear boundaries to seperate data with the SVM, it makes use of kernel functions. These functions create new features of the data using existing features - thus bringing the seperation problem into a new dimensional space. This allows a seperation to be more likely to be found. 

<img src="SVM2.png">

Given a student the SVM uses decision boundaries found with the additional features made from the kernel function to predict if they need or don't need early intervention. 

**Good analogy of SVM:** There are two color balls that need to be seperated (data) so SVM puts as stick between them (classifier) as to have the biggest gap on either side of the stick. We can figure out where to put the stick with just a few of the closest balls (support vectors). If there is no stick that can seperate the balls well, then you can throw the balls in the air (kerneling) and slide a sheet of paper (hyperplane) between the balls as to seperate them. 

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [41]:
# TODO: Import 'gridSearchCV' and 'make_scorer'
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score
from sklearn import svm

def performance_metric(y_true, y_predict):
    error = f1_score(y_true, y_predict, pos_label='yes')
    return error


def fit_model(X, y):
  
    # TODO: Initialize the classifier
    classifier = svm.SVC()

    # TODO: Create the parameters list you wish to tune
    parameters = {'kernel':['poly', 'rbf', 'sigmoid'], 'degree':[1, 2, 3], 'C':[0.1, 1, 10]}


    # TODO: Make an f1 scoring function using 'make_scorer' 
    f1_scorer = make_scorer(performance_metric,
                                   greater_is_better=True)

    # TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
    clf = GridSearchCV(classifier,
                       param_grid=parameters,
                       scoring=f1_scorer)

    # TODO: Fit the grid search object to the training data and find the optimal parameters
    clf.fit(X, y)

    # Return the optimal model
    return clf

clf = fit_model(X_train, y_train)
print "Successfully fit a model."

print "The best parameters were: " 

print clf.best_params_


# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Successfully fit a model.
The best parameters were: 
{'kernel': 'poly', 'C': 1, 'degree': 1}
Made predictions in 0.0056 seconds.
Tuned model has a training F1 score of 0.8283.
Made predictions in 0.0032 seconds.
Tuned model has a testing F1 score of 0.8027.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** The untuned model had a testing F1 score of 0.7660 at a computation time of 0.0028 seconds.

The tuned model had a testing F1 score of 0.8027 at a computation time of 0.0032 seconds.

As the results will show, tuning the model has improved its overall F1 score performance for testing. However, the computation time took slightly longer in the tuned model than the untuned one.



> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.